# Creating and plotting elemental maps

In this tutorial, elemental maps of copper and zinc will be created from electron energy loss spectra (EELS). The data is recorded using a sample of copper and zinc oxide deposited on carbon nanotubes. The particles are likely to be very small, and the carbon is not completely covered. In this sample, the ratio of Zn and Cu is 3:1, and approximately 80 wt% of the sample is carbon. Due to the small amount of copper and zinc, the signal strength is low. To improve the signal strength without lowering the spatial resolution, the spectra were recorded with a high energy dispersion, resulting in low energy resolution. This means that any fine structure is not visible, and Hartree-Slater cross sections will be used to fit Cu and Zn. 

For more information about the material system and how the results of this processing can be used, see the paper: https://doi.org/10.1016/j.cattod.2019.02.045

This notebook requires HyperSpy 1.5 or higher.

The main objective is to show how the functionality of HyperSpy can be used to create elemental maps from EEL spectra.



### Changes

* 2017/09/27: Initial version by Ida Hjorth
* 2019/11/14: Update to HyperSpy 1.5, and minor improvements to text by Magnus Nord

### Table of contents

1. <a href='#load'> Load data</a> 
2. <a href='#background'> Background removal</a>
3. <a href='#fitting'>Fitting Cu and Zn Hartree-Slater cross sections</a>
4. <a href='#save_restore'>Saving and restoring the model</a>
5. <a href='#plotting'>Creating elemental maps from the model</a>

# <a id='load'></a>1. Load data

In [ ]:
%matplotlib notebook
import numpy as np
import hyperspy.api as hs

In [ ]:
s_eels = hs.load('datasets/CuZn_EELS_mapping_tutorial.hspy')

Making elemental maps for this map is a bit tricky, due to the fairly weak edges, and the overlap between the Cu and Zn edges.

In [ ]:
s_eels.plot()

# Generate synthetic zero loss peak for low-loss signal (if lacking)

In the model-based approach, the low-loss signal can be convolved with the high-loss signal such that the energy spread of the electron beam and plural scattering is taken into account. This will lead to a better fit of the model to the experimental data. In addition, the experimental zero loss peak can be used to precisely calibrate the energy scale. If you have recorded a low-loss signal, you can skip this part and go directly to the generation of the background model.

Unfortunately, in our example the low-loss signal was not recorded. Since the core loss components we'll be fitting to data have sharp onsets and sharp features, we make a synthetic low loss signal to take into account the energy spread of the beam.

We give this Gaussian component a full width at half maximum (FWHM) equivalent to approximately the energy spread of the beam, acquired at a fairly high spectrometer dispersion (~4 eV)
and a low loss signal with the same spatial dimensions as the high-loss EELS signal. This is the second best option, after of course having a real low-loss signal.

In [ ]:
g = hs.model.components1D.Gaussian(A=1., centre=0.) # Zero loss peak
g.fwhm = 5

The zero-loss signal `s_zlg` can be generated as follows. This synthetic signal accounts for the energy spread of the electron beam, but not plural scattering as would with a recorded low-loss signal.

In [ ]:
energy_axis = np.arange(-50, 200, 1)
shape = (s_eels.axes_manager.navigation_axes[1].size, s_eels.axes_manager.navigation_axes[0].size, len(energy_axis))
lowloss = np.zeros(shape)
lowloss[:, :] = g.function(energy_axis)
s_zlg = hs.signals.EELSSpectrum(lowloss)
s_zlg.axes_manager.signal_axes[0].offset = -50

# <a id='background'></a>2. Find and subtract the background

The background of the high-loss signal is found by fitting a power law to the signal. If you have recorded a low-loss signal, load it now and insert it in the model rather than the constructed version

In [ ]:
bm = s_eels.create_model(ll=s_zlg) # ll=low-loss signal or constructed zero-loss signal
bm.set_signal_range(750., 915.) # Cu-edge starts at 931 eV

As there are large variations in intensity across the sample, the background is best fitted by giving constraints such as boundary conditions and initial values. The following constraints are found to work well on the dataset used in this tutorial.

In [ ]:
bm.components

In [ ]:
p = bm.components.PowerLaw
p.A.value = 100000000000.
p.A.bmin = 10000000000.
p.A.bmax = 1000000000000.
p.r.value = 2.8
p.r.bmin = 2.5
p.r.bmax = 3.1
bm.components.PowerLaw.A.assign_current_value_to_all() # Sets the value to all the probe positions points (navigation dimension).
bm.components.PowerLaw.r.assign_current_value_to_all()

In [ ]:
bm.multifit(fitter='leastsq', bounded=True)
bm.reset_signal_range()

We now plot the signal and the fitted background model, to see the quality of the fit. Go specifically to point `(31, 14)`, where there is a clear copper and zinc edge. 

In [ ]:
bm.plot() # Check point (31, 14)

The background fits nicely to most of the image, but in vacuum the fitting is not as good (naturally). A trace non-quantifiable Ni-component at about 850 eV also makes the background fitting more challenging.

Let's see how the signal looks like with the background removed.

In [ ]:
s1_eels = s_eels - bm.as_signal()
s1_eels.plot()

# <a id='fitting'></a>3. Fitting Cu and Zn components to the spectrum

A new model consisting of Cu and Zn Hartree-Slater cross sections will be created. For this to work, Hartree-Slater GOS files must configured in HyperSpy.

The path to these cross sections must be set in the preferences, under the EELS tab

In [ ]:
hs.preferences.gui()

Next, the experimental microscope parameters is set, and the model is created. Since the background already has been removed, we use `auto_background=False` when creating the model.

In [ ]:
s1_eels.set_microscope_parameters(beam_energy=200, convergence_angle=13.33, collection_angle=55.28) 
s1_eels.add_elements(('Cu', 'Zn')) 
# Adding Hartree Slater cross sections. For this to work, H-S GOS must be installed.
m = s1_eels.create_model(ll=s_zlg, auto_background=False)

In [ ]:
m.components

Due to the varying and low signal strength, some initial parameters and boundaries are set to ensure successful fitting. 

In [ ]:
m.components.Cu_L3.onset_energy.value = 931
m.components.Zn_L3.onset_energy.value = 1020
m.components.Zn_L3.intensity.bmin = 0 # lower boundary value (negative values are non-physical)
m.components.Zn_L3.intensity.value = 60 # initial value
m.components.Zn_L3.intensity.assign_current_value_to_all() # sets these values to all the scanning points.
m.components.Cu_L3.intensity.bmin = 0
m.components.Cu_L3.intensity.value = 60
m.components.Cu_L3.intensity.assign_current_value_to_all()

We first fit the copper components, as it appears in front of the Zn component. Only the Cu component must then be active, and the signal range is set to a region where only Cu appears.

In [ ]:
# Fitting Cu component first
m.components.Cu_L1.active = True
m.components.Cu_L2.active = True
m.components.Cu_L3.active = True
m.components.Zn_L1.active = False
m.components.Zn_L2.active = False
m.components.Zn_L3.active = False

m.set_signal_range(900., 1000.)
m.multifit(fitter='leastsq', bounded=True) #Using bounded fitting

Now we lock the parameters for the Cu components, and set the Zn components to active. 

Allowing for the Zn to be fitted, while the Cu components remains constant.

In [ ]:
# Zn fitting
m.components.Cu_L3.intensity.free = False

m.set_signal_range(1000., 1300.)
m.components.Zn_L1.active = True
m.components.Zn_L2.active = True
m.components.Zn_L3.active = True

m.multifit(fitter='leastsq', bounded=True)

Lastly, the both the Cu and Zn components are fitted together over the whole energy range.

In [ ]:
m.components.Cu_L3.intensity.free = True
m.reset_signal_range()
m.multifit(fitter='leastsq', bounded=True)

Let's see the results. Go to the region around `(32, 14)`, where there clearly is copper or zinc of some kind.

In [ ]:
m.plot(plot_components=True)

# 4. <a id='save_restore'></a>Saving and restoring the models

As the fitting process can be slow, saving the models can be a good idea. 

In [ ]:
m.save('model.hspy', overwrite=True)

This results in a file called `model.hspy`, which can be loaded and restored.

In [ ]:
mr = hs.load('model.hspy')
mr = mr.models.restore('a')

# 5. <a id='plotting'></a>Make maps

In this section, elemental maps will be plotted from the intensity of the model of the Cu-L$_3$ and Zn-L$_3$ components saved above. The HAADF image will also be plotted, for comparison with the elemental distribution.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
import matplotlib.font_manager as fm

In [ ]:
m = hs.load('model.hspy')
mr = m.models.restore('a')

In [ ]:
s_copper = mr.components.Cu_L3.intensity.as_signal()
s_zinc = mr.components.Zn_L3.intensity.as_signal()
s_haadf = hs.load('datasets/CuZn_HAADF.hspy').as_signal2D((0, 1))

`GridSpec` is a convenient function to use when creating several subplots. We create axes for the elemental maps, HAADF signal as well as the colorbars that will be used indicate the numerical intensity.

In [ ]:
fig = plt.figure(figsize=(6, 2.7))
gs = gridspec.GridSpec(30, 3)
ax_cu = fig.add_subplot(gs[0:-1, 0])
ax_zn = fig.add_subplot(gs[0:-1, 1])
ax_haadf = fig.add_subplot(gs[0:-1, 2])
cbar_cu =  fig.add_subplot(gs[-1, 0]) # Colorbars are much thinner than the map axes (1/30 of the height of the image)
cbar_zn =  fig.add_subplot(gs[-1, 1])
cbar_haadf =  fig.add_subplot(gs[-1, 2])

The signals can be plotted by using imshow. Origin and extent can be changed to alter the ...

In [ ]:
cax_cu = ax_cu.imshow(
    s_copper.data,
    interpolation='nearest',
    origin='lower',
    extent=s_copper.axes_manager.signal_extent
)

In [ ]:
cax_zn = ax_zn.imshow(
    s_zinc.data,
    interpolation='nearest',
    origin='lower',
    extent=s_zinc.axes_manager.signal_extent
)

NumPy functions, such as `flipud`, `fliplr` and `rot90` can also be used to give the image the correct orientation. Here,`flipud` is used to give the HAADF image the same orientation as the elemental maps.

In [ ]:
ax_haadf.imshow(
    np.flipud(s_haadf.data),
    interpolation='nearest',
    origin='upper',
    extent=s_haadf.axes_manager.signal_extent
)

Disable axis ticks

In [ ]:
ax_haadf.set_xticks([])
ax_cu.set_xticks([])
ax_zn.set_xticks([])
ax_zn.set_yticks([])
ax_cu.set_yticks([])
ax_haadf.set_yticks([])

Scalebars can be added by using AnchoredSizeBar. 

In [ ]:
fontprops = fm.FontProperties(size=18)
scalebar0 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1, # length of bar, label, loc
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops)
scalebar1 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops)
scalebar2 = AnchoredSizeBar(
        ax_cu.transData,
        5, '5 nm', 1,
        pad=0.1,
        color='white',
        frameon=False,
        size_vertical=0.6,
        fontproperties=fontprops)
ax_cu.add_artist(scalebar0)
ax_zn.add_artist(scalebar1)
ax_haadf.add_artist(scalebar2)

The color schemes `viridis`, `inferno`, `plasma` and `magma` are nice to use as they are grayscale compatible, perceptually uniform and colorblind-proof. For more info on these color maps, see https://www.youtube.com/watch?v=xAoljeRJ3lU

In [ ]:
cax_zn.set_clim(vmin=0, vmax=30)
cax_zn.set_cmap('inferno')
cax_cu.set_clim(vmin=0, vmax=21)
cax_cu.set_cmap('viridis')

We add labels to indicate what is shown in each image.

In [ ]:
ax_cu.text(0.05,0.05, 'Cu', color='white', size=18, transform=ax_cu.transAxes)
ax_zn.text(0.05,0.05, 'Zn', color='white', size=18, transform=ax_zn.transAxes)
ax_haadf.text(0.05,0.05, 'HAADF', color='white', size=18, transform=ax_haadf.transAxes)

Lastly, the colorbars are added. 

In [ ]:
cb_zn = fig.colorbar(ax_zn.images[0], cax=cbar_zn, extend='both', orientation='horizontal')  
cb_cu = fig.colorbar(ax_cu.images[0], cax=cbar_cu,extend='both', orientation='horizontal')  
cb_haadf = fig.colorbar(ax_haadf.images[0], cax=cbar_haadf, extend='both', orientation='horizontal')

In [ ]:
cb_cu.set_ticks([0, 10, 20, 30, 40])
cb_zn.set_ticks([0, 15, 30, 45, 60])
cb_haadf.set_ticks([0, 1000, 2000])

In [ ]:
gs.update(hspace=0.02, wspace=0.02)

Saving the matplotlib figure object as a png-file

In [ ]:
fig.savefig("elemental_map.png", dpi=300)